In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
# shp = gpd.read_file('municipios240101.geojson')

# Assign random values to a new column
# shp['random'] = np.random.randint(0, 2, size=len(shp))

# Save as geoparquet in 2 part files
# shp[shp['random'] == 0].to_parquet('municipios240101.parquet/part0.parquet', engine='pyarrow')
# shp[shp['random'] == 1].to_parquet('municipios240101.parquet/part1.parquet', engine='pyarrow')

Matplotlib is building the font cache; this may take a moment.


In [74]:
# Create a geopandas dataframe with 100 or 200 linestrings (representing a road network), they should have the following attributes:
# edgeKeyId, osmWayId, startOsmNodeId, endOsmNodeId, functionalClass (1-9, 11-15, 20-24) and highwayType (residential, primary, secondary, tertiary, unclassified, service, living_street, track, footway, path)
# The way to generate the linestrings should be the following: create num random points and create a linestring between some of those randomly
def create_linestrings(num_lines):
    lines = []
    info = []
    # Generar puntos aleatorios en un rango aproximado de coordenadas de España
    x1s, y1s = np.random.uniform(-9.5, 3.5, size=num_lines), np.random.uniform(35.5, 43.5, size=num_lines)
    # Now create a linestring between some of those points joined in random
    for idx in range(num_lines):
        startId = np.random.randint(0, num_lines)
        x1, y1 = x1s[startId], y1s[startId]
        endId = np.random.randint(0, num_lines)
        while startId == endId:
            endId = np.random.randint(0, num_lines)
        
        x2, y2 = x1s[endId], y1s[endId]
        wkt = f"LINESTRING({x1} {y1}, {x2} {y2})"
        lineForward = gpd.GeoSeries.from_wkt([wkt])
        oneway = True

        if np.random.rand() < 0.5:
            # Create a second line with the opposite direction
            wkt = f"LINESTRING({x2} {y2}, {x1} {y1})"
            lineBackward = gpd.GeoSeries.from_wkt([wkt])
            edgeKeyId = idx * 2 + 1
            oneway = False
            lines.append(lineBackward)
            info.append((edgeKeyId, endId, startId, oneway))
    
        edgeKeyId = idx * 2
        lines.append(lineForward)
        info.append((edgeKeyId, startId, endId, oneway))


    
    # Create a dataframe with the linestrings and the attributes
    df = gpd.GeoDataFrame(gpd.GeoSeries(pd.concat(lines).reset_index(drop=True)), columns=['geometry'])
    n_lines = len(df)
    df['edgeKeyId'] = [d[0] for d in info]
    df['osmWayId'] = [int(d[0] / 2) + 100000 for d in info]
    df['startOsmNodeId'] = [d[1] for d in info]
    df['endOsmNodeId'] = [d[2] for d in info]
    df['oneway'] = [d[3] for d in info]
    df['facType'] = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 20], size=n_lines)
    df['highwayType'] = np.random.choice(['residential', 'primary', 'secondary', 'tertiary', 'unclassified', 'service', 'living_street', 'track', 'footway', 'path'], size=n_lines)
    df['osmDirection'] = df['edgeKeyId'].apply(lambda x: 1 if x % 2 == 0 else -1)
    df['flagged'] = df['osmWayId'] % 10 == 0
    
    return df

In [77]:
network = create_linestrings(40)
import os
os.makedirs('test-bucket/network.parquet', exist_ok=True)
network.to_parquet('test-bucket/network2.parquet/part0.parquet', engine='pyarrow')

In [71]:
network[network['oneway'] == False]

,geometry,edgeKeyId,osmWayId,startOsmNodeId,endOsmNodeId,oneway,functionalClass,highwayType,osmDirection
0,"LINESTRING (-7.07341 42.74356, -3.00266 41.05172)",1,100000,21,9,False,6,path,-1
1,"LINESTRING (-3.00266 41.05172, -7.07341 42.74356)",0,100000,9,21,False,2,path,1
2,"LINESTRING (-6.9216 40.14968, 1.60874 35.83082)",3,100001,0,17,False,5,track,-1
3,"LINESTRING (1.60874 35.83082, -6.9216 40.14968)",2,100001,17,0,False,12,tertiary,1
5,"LINESTRING (1.6376 43.38008, -3.72842 37.53142)",7,100003,47,8,False,23,service,-1
6,"LINESTRING (-3.72842 37.53142, 1.6376 43.38008)",6,100003,8,47,False,4,living_street,1
10,"LINESTRING (2.90578 38.96888, -1.1673 42.34897)",15,100007,15,42,False,4,living_street,-1
11,"LINESTRING (-1.1673 42.34897, 2.90578 38.96888)",14,100007,42,15,False,5,footway,1
12,"LINESTRING (-5.30351 42.41419, 1.36535 37.69969)",17,100008,43,45,False,20,path,-1
13,"LINESTRING (1.36535 37.69969, -5.30351 42.41419)",16,100008,45,43,False,3,unclassified,1
